In [17]:
import pandas as pd
import re
from tqdm import tqdm

In [5]:
websites_brut = pd.read_csv('websites.csv')
keywords = pd.read_csv('key_words_formatted.csv')

In [59]:
def filter_texts_by_keywords(texts, keywords, context_radius):
    results = {}
    texts = [t for t in texts if isinstance(t, str)]
    keywords = [str(k) for k in keywords]
    for i, keyword in enumerate(keywords):
        pattern = keyword.replace(' ', '.{0,30}')        
        occurrences = []
        
        for text in texts:
            for match in re.finditer(f'(.{{0,{context_radius}}})({pattern})(.{{0,{context_radius}}})', text, re.IGNORECASE | re.DOTALL):
                occurrences.append(match.group(1) + match.group(2) + match.group(3))        
        results[i] = occurrences

    return results

In [25]:
websites_brut[websites_brut['Index']==0]['Text'].tolist()

["Exanthème de l'enfant - MedG\nLe projet\nDétails du projet\nParticipez !\nToutes les infos\nPour les médecins\nLes Missions MedG\nJe fais un don à l’asso\nFiches à créer\nFiches à relire\nIls ont participé : merci 🙂\nPartenaires et soutiens\nLes fiches\nTous les articles\nProgramme R2C\nFiches par items ECNi\n“Maladie et Grand Syndrome”\n“Orientation Diagnostique”\nArbres décisionnels médicaux\nLes matières\n– Synth. ref. des enseignants –\nAnatomie\nCardiologie\nDermatologie\nEndocrinologie\nGénétique\nGériatrie\nGynéco-obstétrique\nHématologie\nHépato-gastro-entérologie\nImmunologie\nInfectiologie\nLCA / Recherche\nMédecine interne\nNéphrologie\nNeurologie\nNutrition – Sport\nOphtalmologie\nORL – CMF – Stomato\nOrthopédie\nOncologie\nPédiatrie\nPneumologie\nPsychiatrie\nRhumatologie\nSanté Publique\nSémiologie\nThérapeutique\nUrgences\nUrologie\nVasculaire\nInformations\nF.A.Q\nL’asso MedG.fr\nLes Fiches MedG\nFiche MedG : Quésaco ?\nPlan des fiches MGS\nPlan des fiches OD\nRédiger

In [60]:
def build_chunks(keywords, websites_brut):
    chunks = []

    for i in tqdm(range(len(keywords))):
        websites_i = websites_brut[websites_brut['Index']==i]['Text'].tolist()
        keyword_i = keywords.iloc[i][1:].tolist()
        chunks.append(filter_texts_by_keywords(websites_i, keyword_i, 200))

    return chunks


In [61]:
chunks = build_chunks(keywords, websites_brut)

100%|██████████| 103/103 [01:04<00:00,  1.60it/s]


In [69]:
chunks_df = pd.DataFrame(chunks)
chunks_df.to_csv('web_chunks.csv', index=False)